In [1]:
%pylab inline
%matplotlib inline

import sys
import os
import pickle
import math


sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/WebScrapping/")


import paths
import names
import tokenizer
import useful_methods
import my_plot
import scrap_espn_gamecast
import espn_urls

Populating the interactive namespace from numpy and matplotlib


### Step 1: Data, Game Infos

In [2]:
# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

dfGameInfos.head()

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
0,1,Tottenham,Sat. 8 Aug.,United,0,1,0,1,11:45
1,1,Watford,Sat. 8 Aug.,Everton,2,2,1,0,14:00
2,1,Sunderland,Sat. 8 Aug.,Leicester,2,4,0,3,14:00
3,1,Crystal,Sat. 8 Aug.,Norwich,3,1,1,0,14:00
4,1,Villa,Sat. 8 Aug.,Bournemouth,1,0,0,0,14:00


In [3]:
weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home']) 
score_ft_aways = list(dfGameInfos['score_ft_away'])

### Step 2: Load Naive Bayes Classifier

In [4]:
# Read Naive Bayes Classifier
with open(paths.READ_PATH_REVIEW_SHORT + 'nb_reviews_sentiment_detector.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'utf-8'
    nb_detector_reloaded = u.load()

classifier = nb_detector_reloaded

### Step 3: Classify All Games by Naive Bayes

In [5]:
# Tweet Classifier
# Return POS, NEG %
def Classify(dfFilter, side, start=1, end=63):
    
    # set time intervals
    df = dfFilter[(dfFilter['side'] == side) &
                  (dfFilter['ith_minute'] >= start) & (dfFilter['ith_minute'] <= end)].copy()
    
    # Classify teach text and save scores
    texts = list(df['text'])
    nb_negs, nb_poss = [], []

    for text in texts:
        # classify text
        nb_neg, nb_pos = classifier.predict_proba([text])[0] # scoring here

        # save results
        nb_negs.append(nb_neg)
        nb_poss.append(nb_pos)
    
    
    # Sum all scores
    nb_pos_sum = sum(nb_poss)
    nb_neg_sum = sum(nb_negs)
    
    # Return POS, NEG %
    nb_pos_perc = nb_pos_sum / (nb_pos_sum + nb_neg_sum)
    nb_neg_perc = nb_neg_sum / (nb_pos_sum + nb_neg_sum)
    
    return nb_pos_perc, nb_neg_perc

In [6]:
# Get Single Game Datas
def EplClassifyAll():
    # results container
    res_ht_nb = []
    
    # run through each games
    for index in range(len(dfGameInfos)):
        
        try:
            # Read Game as DF
            week = weeks[index]
            team_home = home_teams[index]
            team_away = away_teams[index]

            os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(week) + '/SingleGames')
            df = useful_methods.csv_dic_df(team_home + "_vs_" + team_away + ".csv")

            # Filtering. Removing bots
            dfFilter = useful_methods.FilterDF(df)
            dfFilter['ith_minute'] = [int(ith_minute) for ith_minute in dfFilter['ith_minute']]

            # Classify Texts
            nb_pos_home, nb_neg_home = Classify(dfFilter, 'home')
            nb_pos_away, nb_neg_away = Classify(dfFilter, 'away')
            print(week, nb_pos_home, nb_neg_home, nb_pos_away, nb_neg_away)
            
            # Append Result
            res_ht_nb.append([index, nb_pos_home, nb_neg_home, nb_pos_away, nb_neg_away])
        
        except:
            continue
            
    return res_ht_nb

In [7]:
%time res_epl_nb = EplClassifyAll()

4 0.466622678097 0.533377321903 0.471093692326 0.528906307674
4 0.437947898351 0.562052101649 0.453864335353 0.546135664647
4 0.495660906507 0.504339093493 0.501924098069 0.498075901931
4 0.452181912279 0.547818087721 0.481851016599 0.518148983401
4 0.502295796979 0.497704203021 0.411277725998 0.588722274002
4 0.480030774349 0.519969225651 0.465106600855 0.534893399145
4 0.501096430991 0.498903569009 0.478419737694 0.521580262306
4 0.504249211791 0.495750788209 0.430386375721 0.569613624279
4 0.452540661731 0.547459338269 0.476341517369 0.523658482631
5 0.491206937507 0.508793062493 0.425470643838 0.574529356162
5 0.496682534176 0.503317465824 0.465423305334 0.534576694666
5 0.452883177941 0.547116822059 0.452497162848 0.547502837152
5 0.462171741997 0.537828258003 0.417346886763 0.582653113237
5 0.507193230403 0.492806769597 0.465024960254 0.534975039746
5 0.484404123492 0.515595876508 0.477790416091 0.522209583909
5 0.438717103046 0.561282896954 0.4372985167 0.5627014833
5 0.48833605

In [8]:
# Save Resulst as CSV
dfResult = dfGameInfos.copy()

def PnScore(index, side):
    for res in res_epl_nb:
        if index == res[0]:
            if not math.isnan(res[side]):
                return res[side]
    return -1

dfResult['nb_pos_home'] = [PnScore(index, 1) for index in range(len(weeks))]
dfResult['nb_neg_home'] = [PnScore(index, 2) for index in range(len(weeks))]
dfResult['nb_pos_away'] = [PnScore(index, 3) for index in range(len(weeks))]
dfResult['nb_neg_away'] = [PnScore(index, 4) for index in range(len(weeks))]



# Save as CSV
useful_methods.DFtoCSV(dfResult, paths.READ_PATH_GAME_INFO, 'Predict-Review-63min', index=False)
print("[Saved in]: %s" % (paths.READ_PATH_GAME_INFO + 'Predict-Review-63min.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/Predict-Review-63min.csv


In [9]:
len(dfResult[dfResult['nb_neg_home'] != -1])

156